In [1]:
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,toliara,-23.3500,43.6667,25.90,82,100,1.30,MG,1710107190
1,1,birao,10.2849,22.7882,27.08,11,29,2.39,CF,1710107190
2,2,lihue,21.9789,-159.3672,25.58,77,100,7.72,US,1710107190
3,3,port-aux-francais,-49.3500,70.2167,7.51,92,97,14.61,TF,1710107191
4,4,puerto natales,-51.7236,-72.4875,10.25,50,40,11.83,CL,1710107191


In [3]:
crit_df=city_data_df.loc[(city_data_df['Max Temp']>21) & (city_data_df['Max Temp']<30) & 
                         (city_data_df['Humidity']<80) & (city_data_df['Wind Speed']<4)
                        &(city_data_df['Cloudiness']<75)].dropna()


# Display sample data
crit_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,birao,10.2849,22.7882,27.08,11,29,2.39,CF,1710107190
47,47,bosaso,11.2842,49.1816,27.03,76,1,0.80,SO,1710107213
62,62,kingston,17.9970,-76.7936,29.53,68,10,2.53,JM,1710107219
69,69,adamstown,-25.0660,-130.1015,24.29,68,32,2.03,PN,1710107223
118,118,tocopilla,-22.0920,-70.1979,21.95,73,67,2.34,CL,1710106913
137,137,seka,17.9285,103.9552,22.60,74,9,2.03,TH,1710107256
138,138,tshabong,-26.0500,22.4500,26.37,13,0,3.90,BW,1710107257
160,160,ta`u,-14.2336,-169.5144,28.62,78,47,3.49,AS,1710107267
177,177,hithadhoo,-0.6000,73.0833,28.18,71,33,3.60,MV,1710107274
190,190,enriquillo,17.9000,-71.2333,28.82,67,3,2.96,DO,1710107281


In [4]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=crit_df.loc[:,['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''



# Display sample data
hotel_df.head(10)

,City,Country,Lat,Lng,Humidity,Hotel Name
1,birao,CF,10.2849,22.7882,11,
47,bosaso,SO,11.2842,49.1816,76,
62,kingston,JM,17.9970,-76.7936,68,
69,adamstown,PN,-25.0660,-130.1015,68,
118,tocopilla,CL,-22.0920,-70.1979,73,
137,seka,TH,17.9285,103.9552,74,
138,tshabong,BW,-26.0500,22.4500,13,
160,ta`u,AS,-14.2336,-169.5144,78,
177,hithadhoo,MV,-0.6000,73.0833,71,
190,enriquillo,DO,17.9000,-71.2333,67,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [5]:
# base_url="https://api.geoapify.com/v2/places"
# radius = 10000
# categories='accommodation.hotel'
# latitude=hotel_df.loc[62,'Lat']
# longitude=hotel_df.loc[62,'Lng']
# limit=1

# filters = f"circle:{longitude},{latitude},{radius}"
# bias = f"proximity:{longitude},{latitude}"
# params = {'categories':categories,'filter':filters,'bias':bias,'apiKey':geoapify_key,'limit':limit
    
# }
# hotel_data=requests.get(base_url,params)
# print(hotel_data.url)
# radius = 10000
# categories='accommodation.hotel'
# params = {
    
# }

In [6]:
radius = 10000
categories='accommodation.hotel'
limit=1
# filters = f"circle:{longitude},{latitude},{radius}"
# bias = f"proximity:{longitude},{latitude}"
params = {'categories':categories,
          'apiKey':geoapify_key,'limit':limit}

print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude=hotel_df.loc[index,'Lat']
    longitude=hotel_df.loc[index,'Lng']
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params)
    
    # Convert the API response to JSON format
    name_address = requests.get(base_url,params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
birao - nearest hotel: No hotel found
bosaso - nearest hotel: Bosaso hotel
kingston - nearest hotel: Aphrodite Hotel


KeyboardInterrupt: 